In [42]:
import pandas as pd
import numpy as np
from glob import glob
from tqdm.notebook import tqdm
from collections import Counter
import json

In [88]:
def get_df_PER(df):
    df_PER_1 = df[df['Tag'] == 'B-PER']
    df_PER_2 = df[df['Tag'] == 'S-PER']
    
    df_PER = pd.concat([df_PER_1, df_PER_2])
    df_PER = df_PER.sort_index(ascending=True)
    return df_PER

In [153]:
def get_counter(list_tag, N):
    list_elem_cleared = [elem for elem in list_tag if elem is not None]
    return Counter(list_elem_cleared).most_common(N+1)

In [122]:
def get_N_surronding_tokens(index, df_courant, N):
    list_token = []
    list_mention_token = []
    list_head_token = []
    
    for index_mention in index:
        if index_mention > N:
            for i in range(index_mention-N, index_mention+N):
                if i < len(df_courant):
                    if df_courant.loc[[i]]['Token'].values[0] is not None:
                        list_token.append(df_courant.loc[[i]]['Token'].values[0])
                        list_mention_token.append(df_courant.loc[[i]]['Tag'].values[0])
                        list_head_token.append(df_courant.loc[[i]]['Head_tag'].values[0])
        else:
            for i in range(index_mention, index_mention+N):
                if df_courant.loc[[i]]['Token'].values[0] is not None:
                    list_token.append(df_courant.loc[[i]]['Token'].values[0])
                    list_mention_token.append(df_courant.loc[[i]]['Tag'].values[0])
                    list_head_token.append(df_courant.loc[[i]]['Head_tag'].values[0])

    return list_token, list_mention_token, list_head_token

In [132]:
def get_PER_surrounding(list_mention_token, list_head_token):
    list_surronding_PER = []
    for i in range(len(list_mention_token)):
        if list_mention_token[i] == 'B-PER' or list_mention_token[i] == 'S-PER':
            list_surronding_PER.append(list_head_token[i])
    return list_surronding_PER

In [125]:
def get_dict_surronding_PER(df_courant, N_PER, N_token):
    dict_surronding_PER = {}
    df_PER = get_df_PER(df_courant)
    dict_mention = get_counter(list(df_PER['Head_tag']), N_PER)
    df_mention_roman = pd.DataFrame(dict_mention, columns=['mention', 'count'])
    list_mention = list(df_mention_roman['mention'])
    for i in range(1, len(list_mention)):# exit nan value
        # get_index_mention w/ PER preds
        df_mentionned = df_courant[df_courant['Head_tag'] == list_mention[i]]
        index = df_mentionned.index
        # get N surronding tokens (useful for topic modelling, later), mention_tokens (BIES), head_mention_tokens (head coref mention)
        list_surronding_token, list_surronding_mention_token, list_surronding_head_token = get_N_surronding_tokens(index, df_courant, N_token)
        # get head mention if mention is B or S-PER 
        list_surronding_PER = get_PER_surrounding(list_surronding_mention_token, list_surronding_head_token)
        # TODO - WHY SO MUCH NAN VALUES ? It is supposed to be only PER values
        dict_surronding_PER[list_mention[i]+'_'+str(i)] = list_surronding_PER
    return dict_surronding_PER         

In [149]:
def mouli_preds(path_name, N_PER, N_token):
    str_base = '_'
    for doc in tqdm(glob(path_name)):
        print(doc)
        df_courant = pd.read_csv(doc)
        dict_surronding_PER = get_dict_surronding_PER(df_courant, N_PER, N_token)

        temp = doc.split('\\')[1].split('.')[0]
        
        df_surronding_PER = pd.DataFrame.from_dict(dict_surronding_PER, orient='index')
        df_surronding_PER = df_surronding_PER.transpose()
        df_surronding_PER.to_csv(r'res/'+temp+'.csv',  encoding='utf-8', index=False)
        
    return df_surronding_PER

In [150]:
path_name = "data/*.csv"
path_test = "data_test/*.csv"
N_PER = 10
N_token = 50

In [151]:
df_test = mouli_preds(path_test, N_PER, N_token)

  0%|          | 0/1 [00:00<?, ?it/s]

data_test\Balzac-Honore-de_Eugenie-Grandet.csv


In [152]:
df_test

,<Eugénie-3301>_1,<le père-2169>_2,<monsieur Grandet-943>_3,<Grandet au Tribunal de Première Instance-2317>_4,<Eugénie-1205>_5,<Eugénie et sa mère-1499>_6,<Eugénie-1353>_7,<un homme-191>_8,<l'-2932>_9
0,<Des Grassins-3288>,<lui-2132>,<madame des Grassins-921>,<- ils-2310>,<du tonnelier-1197>,<Charles-1473>,<Il-1350>,<du joueur-189>,<Monsieur Bergerin-2927>
1,<Charles-3249>,<lui-2132>,<madame des Grassins-921>,<leur-2313>,<Eugénie-1205>,<Charles-1473>,<Monsieur Grandet-1333>,<du courtisan-190>,<Eugénie-2902>
2,<un homme-3298>,<un enfant-2168>,<est-940>,<des Grassins-2315>,<Eugénie-1205>,<Charles-1473>,<Monsieur Grandet-1333>,<monsieur Grandet-112>,<Monsieur Bergerin-2927>
3,<un homme-3298>,<un enfant-2168>,<les uns-941>,<Charles-2316>,<Eugénie-1205>,<Charles-1473>,<du notaire-1329>,<monsieur Grandet-112>,<Monsieur Bergerin-2927>
4,<Charles-3249>,<le père-2169>,<les autres-942>,<Grandet au Tribunal de Première Instance-2317>,<Eugénie-1205>,<Charles-1473>,<du notaire-1329>,<un homme-191>,<Eugénie-2902>
...,...,...,...,...,...,...,...,...,...
4728,<mademoiselle Eugénie-3428>,None,None,None,None,None,None,None,None
4729,<Nanon-3433>,None,None,None,None,None,None,None,None
4730,<mademoiselle Eugénie-3428>,None,None,None,None,None,None,None,None
4731,<les habitués de son cercle-3441>,None,None,None,None,None,None,None,None
